In [1]:
import librosa
import pandas as pd
import numpy as np

In [2]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

# Tempo prediction with XGBoost

Since CatBoost isn't working with the app, let's try XGBoost

In [3]:
%run ../Utilities/Utilities.ipynb

Using TensorFlow backend.


In [4]:
data = pd.read_pickle("../Data/Guitar/bpm-data.pkl")

In [5]:
X_train, y_train, X_val, y_val, X_test, y_test = split_data(data, "bpm")

In [6]:
model = XGBRegressor(objective="reg:squarederror",
                     max_depth=10,
                     learning_rate=0.01,
                     reg_lambda=2,
                     n_estimators=500,
                     n_jobs=-1,
                     tree_method="gpu_hist")

In [7]:
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=True, eval_metric="mae")

E:\Anaconda\envs\tfm\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
E:\Anaconda\envs\tfm\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[0]	validation_0-mae:135.912
[1]	validation_0-mae:134.556
[2]	validation_0-mae:133.216
[3]	validation_0-mae:131.885
[4]	validation_0-mae:130.569
[5]	validation_0-mae:129.266
[6]	validation_0-mae:127.974
[7]	validation_0-mae:126.698
[8]	validation_0-mae:125.432
[9]	validation_0-mae:124.18
[10]	validation_0-mae:122.939
[11]	validation_0-mae:121.712
[12]	validation_0-mae:120.495
[13]	validation_0-mae:119.293
[14]	validation_0-mae:118.103
[15]	validation_0-mae:116.922
[16]	validation_0-mae:115.757
[17]	validation_0-mae:114.6
[18]	validation_0-mae:113.457
[19]	validation_0-mae:112.323
[20]	validation_0-mae:111.203
[21]	validation_0-mae:110.092
[22]	validation_0-mae:108.992
[23]	validation_0-mae:107.906
[24]	validation_0-mae:106.829
[25]	validation_0-mae:105.763
[26]	validation_0-mae:104.707
[27]	validation_0-mae:103.662
[28]	validation_0-mae:102.628
[29]	validation_0-mae:101.603
[30]	validation_0-mae:100.59
[31]	validation_0-mae:99.5869
[32]	validation_0-mae:98.5941
[33]	validation_0-mae:97

[269]	validation_0-mae:13.6307
[270]	validation_0-mae:13.5589
[271]	validation_0-mae:13.4878
[272]	validation_0-mae:13.4182
[273]	validation_0-mae:13.3489
[274]	validation_0-mae:13.2811
[275]	validation_0-mae:13.2134
[276]	validation_0-mae:13.1475
[277]	validation_0-mae:13.0816
[278]	validation_0-mae:13.0169
[279]	validation_0-mae:12.9527
[280]	validation_0-mae:12.8891
[281]	validation_0-mae:12.8262
[282]	validation_0-mae:12.7642
[283]	validation_0-mae:12.7031
[284]	validation_0-mae:12.6426
[285]	validation_0-mae:12.5833
[286]	validation_0-mae:12.5247
[287]	validation_0-mae:12.4665
[288]	validation_0-mae:12.409
[289]	validation_0-mae:12.3528
[290]	validation_0-mae:12.2969
[291]	validation_0-mae:12.2417
[292]	validation_0-mae:12.1873
[293]	validation_0-mae:12.1336
[294]	validation_0-mae:12.0809
[295]	validation_0-mae:12.0284
[296]	validation_0-mae:11.977
[297]	validation_0-mae:11.9263
[298]	validation_0-mae:11.8766
[299]	validation_0-mae:11.8274
[300]	validation_0-mae:11.7783
[301]	vali

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.01, max_delta_step=0,
       max_depth=10, min_child_weight=1, missing=None, n_estimators=500,
       n_jobs=-1, nthread=None, objective='reg:squarederror',
       random_state=0, reg_alpha=0, reg_lambda=2, scale_pos_weight=1,
       seed=None, silent=None, subsample=1, tree_method='gpu_hist',
       verbosity=1)

In [8]:
mean_absolute_error(y_train, model.predict(X_train))

7.4740253406148645

In [9]:
mean_absolute_error(y_val, model.predict(X_val))

8.581069915289266

In [10]:
model.predict(pd.DataFrame({
    "d0": [0.475],
    "d1": [0.28125],
    "d2": [0.5125],
    "d3": [0.21875],
    "d4": [0.28125],
    "d5": [0.275],
    "d6": [0.45625],
    "d7": [0.15625]
}))

array([120.57723], dtype=float32)

Saving and loading

In [11]:
model.save_model("tempo_xgboost_model.bin")

In [18]:
loaded = XGBRegressor()

In [19]:
loaded.load_model("tempo_xgboost_model.bin")

In [20]:
loaded.predict(pd.DataFrame({
    "d0": [0.475],
    "d1": [0.28125],
    "d2": [0.5125],
    "d3": [0.21875],
    "d4": [0.28125],
    "d5": [0.275],
    "d6": [0.45625],
    "d7": [0.15625]
}))

array([120.57723], dtype=float32)